<a href="https://colab.research.google.com/github/tommycon/Deeplearning-GoogleCloud/blob/main/AI-Platform/03_Deploy_Model_to_AI_Platform_and_Serve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

chown: invalid user: ‘jupyter:jupyter’


In [ ]:
# Ensure the right version of Tensorflow is installed.
!pip freeze | grep tensorflow==2.1

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# change these to try this notebook out
BUCKET = 'dataflow-tfx-pipeline'
PROJECT = 'tc-test-project-260312'
REGION = 'us-central1'

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [ ]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.1'  

<h2> Deploy trained model </h2>
<p>
Deploying the trained model to act as a REST web service is a simple gcloud call.

In [ ]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model/

gs://dataflow-tfx-pipeline/babyweight/trained_model_keras/20200827113425/
gs://dataflow-tfx-pipeline/babyweight/trained_model_keras/20200827113425/saved_model.pb
gs://dataflow-tfx-pipeline/babyweight/trained_model_keras/20200827113425/assets/
gs://dataflow-tfx-pipeline/babyweight/trained_model_keras/20200827113425/variables/
gs://dataflow-tfx-pipeline/babyweight/trained_model_keras/
gs://dataflow-tfx-pipeline/babyweight/trained_model_keras/20200827113425/
gs://dataflow-tfx-pipeline/babyweight/trained_model_keras/checkpoints/
gs://dataflow-tfx-pipeline/babyweight/trained_model/
gs://dataflow-tfx-pipeline/babyweight/trained_model/checkpoint
gs://dataflow-tfx-pipeline/babyweight/trained_model/events.out.tfevents.1598516366.gke-cml-0827-081152--n1-standard-4-0a-34315352-khp9
gs://dataflow-tfx-pipeline/babyweight/trained_model/graph.pbtxt
gs://dataflow-tfx-pipeline/babyweight/trained_model/model.ckpt-0.data-00000-of-00002
gs://dataflow-tfx-pipeline/babyweight/trained_model/model.ckpt-0.data

In [ ]:
%%bash
MODEL_NAME="babyweight"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/babyweight/trained_model/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ai-platform models delete ${MODEL_NAME}
#gcloud ai-platform models create ${MODEL_NAME} --regions $REGION
gcloud ai-platform versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Deleting and deploying babyweight keras_on_gcpv2 from gs://dataflow-tfx-pipeline/babyweight/trained_model_keras/20200827113425 ... this will take a few minutes


Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......
............................................................................................................................................................................................................................................................................done.


In [ ]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = 'babyweight'
MODEL_VERSION = 'keras_on_gcpv2'

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
    {
      'key': 'b1',
      'is_male': 'True',
      'mother_age': 26.0,
      'plurality': 'Single(1)',
      'gestation_weeks': 39
    },
    {
      'key': 'g1',
      'is_male': 'False',
      'mother_age': 29.0,
      'plurality': 'Single(1)',
      'gestation_weeks': 38
    },
    {
      'key': 'b2',
      'is_male': 'True',
      'mother_age': 26.0,
      'plurality': 'Triplets(3)',
      'gestation_weeks': 39
    },
    {
      'key': 'u1',
      'is_male': 'Unknown',
      'mother_age': 29.0,
      'plurality': 'Multiple(2+)',
      'gestation_weeks': 38
    },
  ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{"error": "Prediction failed: Unexpected tensor name: key"}'


In [ ]:
%%writefile inputs.json
{"key": "b1", "is_male": "True", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}
{"key": "g1", "is_male": "False", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}

Overwriting inputs.json


In [ ]:
%%bash
INPUT=gs://${BUCKET}/babyweight/batchpred/inputs.json
OUTPUT=gs://${BUCKET}/babyweight/batchpred/outputs
gsutil cp inputs.json $INPUT
gsutil -m rm -rf $OUTPUT 
gcloud ai-platform jobs submit prediction babypred_$(date -u +%y%m%d_%H%M%S) \
  --data-format=TEXT --region ${REGION} \
  --input-paths=$INPUT \
  --output-path=$OUTPUT \
  --max-worker-count=1 \
  --model=babyweight --version=ml_on_gcp

jobId: babypred_200827_120725
state: QUEUED


Copying file://inputs.json [Content-Type=application/json]...
-
Operation completed over 1 objects/204.0 B.                                      
Removing gs://dataflow-tfx-pipeline/babyweight/batchpred/outputs/prediction.results-00000-of-00001#1598525394663011...
Removing gs://dataflow-tfx-pipeline/babyweight/batchpred/outputs/prediction.errors_stats-00000-of-00001#1598525386832940...
/ [2/2 objects] 100% Done                                                       
Operation completed over 2 objects.                                              
Job [babypred_200827_120725] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe babypred_200827_120725

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs babypred_200827_120725


In [ ]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/batchpred/outputs

gs://dataflow-tfx-pipeline/babyweight/batchpred/outputs/prediction.errors_stats-00000-of-00001
gs://dataflow-tfx-pipeline/babyweight/batchpred/outputs/beam-temp-prediction.results-f00ed8bae85211eab27542010a800008/
